# Import Libraries

In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from art.attacks.evasion import LowProFool
from art.estimators.classification.scikitlearn import ScikitlearnLogisticRegression

# Load Data

In [48]:
iris = load_iris()

In [49]:
X, y = iris.data, iris.target

In [50]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

In [51]:
scaler = StandardScaler()

In [52]:
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Model

In [53]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

LogisticRegression()

In [54]:
y_pred = lr.predict(x_test)

In [55]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.8666666666666667


In [56]:
classifier = ScikitlearnLogisticRegression(
    model=lr, 
    clip_values=[-2.38621423, 3.03167932]
)

# Attack

In [57]:
attack = LowProFool(
    classifier=classifier,
    n_steps=10,
    eta=10,
    lambd=1.65,
    eta_decay=0.95
)

In [58]:
attack.fit_importances(x_train, y_train)

LowProFool(n_steps=10, threshold=0.5, lambd=1.65, eta=10, eta_decay=0.95, eta_min=1e-07, norm=2, importance=pearson, verbose=False, )

In [59]:
n_classes = len(np.unique(y_train))
n_classes

3

In [74]:
adv_test = attack.generate(x_test, y=np.eye(n_classes)[y_test])

In [75]:
adv_pred = np.argmax(classifier.predict(adv_test), axis=1)

# Results

In [76]:
print("Accuracy:", np.sum(y_test == adv_pred) / len(y_test))

Accuracy: 1.0
